## Load the data

In [ ]:
from pandas import read_csv
import pandas as pd
from pandas import datetime
from matplotlib import pyplot
from matplotlib.pylab import rcParams
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import matplotlib.mlab as mlab

rcParams['figure.figsize'] = 15, 6

dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
elastic = read_csv('/Users/Helen/Documents/Data/codes/pre.csv', parse_dates=['week'],
                   index_col=['week'],date_parser=dateparse) # Load the data of hayfever estimating results from Elastic net
true = read_csv('/Users/Helen/Documents/Data/codes/true.csv', parse_dates=['week'],
                   index_col=['week'],date_parser=dateparse) # Actual RCGP data

print(elastic.head())
print(true.head())
elastic.plot()
true.plot()
pyplot.show()

## Distribute the data size

In [ ]:

# create 4 ARIMAX models by combining estimating results from Elastic net
# 52-week

train, test = elastic[52:104], true[52:104] # test period 1


#train, test = elastic[104:156], true[104:156] # test period 2


#train, test= elastic[156:208], true[156:208] # test period 3

#train, test = elastic[208:261], true[208:261]# test period 4
          

L=test.index
L

In [ ]:
test_=test.values

## ARIMAX

In [ ]:
import statsmodels.api as sm
mod = sm.tsa.SARIMAX(endog = test_,exog = train, order=(5,1,1))#seasonal_order=(5, 1, 1, 1))
fit_res = mod.fit(disp=0)
print(fit_res.summary())
predictions=fit_res.predict()

### RMSE

In [ ]:
df = pd.DataFrame(predictions,index=L,columns=['predictive_rate'])
#df = pd.DataFrame(data=y_pred_enet, index=L,columns=["predicted rate"])
df_true = pd.DataFrame(test_, index=L,columns=["real hayfever rate"])
from sklearn.metrics import mean_squared_error, make_scorer
MSE1 = mean_squared_error(test_, predictions, sample_weight=None, multioutput='uniform_average')
rmse_c=np.sqrt(MSE1)
print("%.4f" % rmse_c)

## Plot

In [ ]:
plt.figure(figsize=(8,5))
plt.ylim((-5, 150))
line_up, = plt.plot(df, label='ARIMA + Elastic net',color='r',linestyle='-')
line_down, = plt.plot(df_true, label='True Hayfever Rate')
plt.legend(handles=[line_up, line_down])
plt.ylabel("Hayfever rate per 100,000 people")
plt.xlabel("Time(week)")
#plt.title('MAE: %.4f'% MAE )
plt.title('RMSE: %.4f'% rmse_c )
pyplot.show()
#pyplot.plot(test)
#pyplot.plot(predictions, color='red')
#pyplot.show()

### MAE

In [ ]:
from sklearn.metrics import mean_absolute_error
mae_c = mean_absolute_error(test, predictions, sample_weight=None, multioutput='uniform_average')
print("%.4f" % mae_c)

## r

In [ ]:
from scipy.stats.stats import pearsonr   
r_c=pearsonr(list(test['hayfever rate']), list(predictions))
print(r_c)

### MAPE

In [ ]:
l1=list(test['hayfever rate'])
l2=list(predictions)
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean((np.abs((np.array(y_true) - np.array(y_pred)) / np.array(y_true)) * 100))
mape_c = mean_absolute_percentage_error(l1, l2)
print("%.8f" % mape_c)

## Elastic net

In [ ]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
data=pd.read_csv('/Users/Helen/Documents/Data/query1/features.csv',parse_dates=['week'],index_col=0,date_parser=dateparse)

In [ ]:

train2, valid2 = data[0:448], data[448:500] #2
#train2, valid2 = data[0:500], data[500:552] #3
#train2, valid2 = data[0:552], data[552:604]#4
#train2, valid2 = data[0:604], data[604:657] #5
len(valid2)

In [ ]:
L2=valid2.index
L2

In [ ]:
features=['hayfever','hay fever','symptoms of hayfever','hayfever remedies','hayfever injection','hayfever cure', 'hayfever treatment',
          'hayfever uk','hayfever relief','hay fever symptoms',
          'hayfever symptoms','hayfever tablets',
          'pollen','hayfever eyes','hayfever eye', 'pollen count','hayfever spray',
          'hayfever count','hayfever medicine',
          'hayfever boots','what is hayfever',
          'hayfever medication','pregnant hayfever','hayfever pregnancy','hayfever in pregnancy',
          'hayfever cures','cures for hayfever',
          'hayfever eye drops','best hayfever tablets','hayfever children',
          'cure hayfever', 'hayfever remedy',
           'itchy eyes','sneezing','red eyes','blocked sinuses','runny nose','shortness of breath',
            'hayfever pollen count',
           'boots hayfever','honey hayfever','bbc pollen count','uk pollen count',
            'remedies for hayfever']
queries=features[0:]
len(set(queries))

In [ ]:
from scipy.stats.stats import pearsonr  
selected=[]
for x in features:
    r=pearsonr(data[x], data['hayfever_rate'])
    if(r[0])>=(0.65):
        selected.append(x)
len(set(selected))

In [ ]:
y_train = train2['hayfever_rate'].values
x_train = train2[selected]
x_valid = valid2[selected]
y_valid = valid2['hayfever_rate'].values

In [ ]:
from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=124.78, l1_ratio=0.5,fit_intercept=True, max_iter=20000, tol=0.0000001) 
#enet = ElasticNet(alpha=87.74, l1_ratio=0.5,fit_intercept=True, max_iter=20000, tol=0.0000001) 
#enet = ElasticNet(alpha=154.01, l1_ratio=0.5,fit_intercept=True, max_iter=20000, tol=0.0000001) 
#enet = ElasticNet(alpha=220.28, l1_ratio=0.5,fit_intercept=True, max_iter=20000, tol=0.0000001) 
enet = enet.fit(x_train, y_train)
y_pred_enet = enet.predict(x_valid)
#enet.intercept_

In [ ]:
df2 = pd.DataFrame(data=y_pred_enet, index=L2,columns=["predicted rate"])
rmse_e=np.sqrt(sum((y_pred_enet-y_valid)**2)/len(y_valid))
rmse_e

## Performance comparison 

In [ ]:
plt.figure(figsize=(9,5))
plt.ylim((-5, 150))
plt.ylabel("Hayfever incidence rate per 100,000 people")
plt.xlabel("Test time(week)")
plt.plot(df2, label='Elastic Net',color='g',linestyle='--')
plt.plot(df, label='ARIMAX with Elastic net',color='r')
plt.plot(df_true, label='True Hayfever Rate')

#plt.title('Test period B')
#plt.suptitle('RMSE of ARIMAX + Elastic net: %.4f'% rmse_c )
plt.legend(['Elastic Net', 'ARIMAX + Elastic net', 'True Hayfever Rate','ARIMA'], loc='upper left')
pyplot.show()